# Projekt - 2. část

Upozornění: Otestována správná funkcionalita vytvořeného Jupyter notebooku s následujícími verzemi knihoven:
* **seaborn: 0.13.0**
* **matplotlib: 3.8.0**
* **pandas: 2.1.1**
* **numpy: 1.24.3**

Se staršími verzemi knihoven není zaručeno validní spuštění.

---

Cíle 2. části projektu jsou:
* provedení explorativní analýzy na zvolené datové sadě
* úpravu datové sady do podoby vhodné pro dolování

## 1. Úkol
### Zadání: 
Z dostupných datových sad si zvolte jednu datovou sadu, kterou se budete dále zabývat. Stáhněte si zvolenou datovou sadu z uvedeného zdroje a prostudujte si dostupné informace k této datové sadě.

### Řešení: 
Vybraná datová sada Most Streamed Spotify Songs 2023 – dostupná zde: [https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023](https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023)

In [ ]:
# Library import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('spotify-2023.csv', encoding='ISO-8859-1')

df.head()

## 2. Úkol
### Zadání: 
Proveďte explorativní analýzu zvolené datové sady. Pro každý následující bod implementujte odpovídající sekci ve zdrojovém kódu a zjištěné výsledky popište v dokumentaci:
1. prozkoumejte jednotlivé atributy datové sady, jejich typ a hodnoty, kterých nabývají (počet hodnot, nejčastější hodnoty, rozsah hodnot atd.)
2. prozkoumejte rozložení hodnot jednotlivých atributů pomocí vhodných grafů, zaměřte se i na to, jak hodnota jednoho či dvou atributů ovlivní rozložení hodnot jiného atributu. Do dokumentace vložte alespoň 5 různých grafů, zobrazujících zjištěná rozložení hodnot. Použijte různé typy grafů (např. bodový graf, histogram, krabicový nebo houslový graf, graf složený z více podgrafů apod.) a věnujte se různým atributům. V dokumentaci také všechny grafy vhodně okomentujte – popište, jaké informace z nich můžeme vyčíst.
3. zjistěte, zda zvolená datová sada obsahuje nějaké odlehlé hodnoty. V dokumentaci popište, jakým způsobem jste odlehlé hodnoty detekovali, a jaké hodnoty jste objevili.
4. proveďte podrobnou analýzu chybějící hodnot. V dokumentaci popište celkový počet chybějících hodnot, počet objektů s více chybějícími hodnotami atd.
5. proveďte korelační analýzu numerických atributů (k analýze využijte grafy i korelační koeficienty).

### Řešení:

#### 1. Podúkol:
Prozkoumejte jednotlivé atributy datové sady, jejich typ a hodnoty, kterých nabývají (počet hodnot, nejčastější hodnoty, rozsah hodnot atd.)

Analýza jednotlivých atributů jako celku:

In [ ]:
df.info()

In [ ]:
numerical_cols = df.select_dtypes(include=['int64']).columns
numerical_data = df[numerical_cols]

descriptive_stats = numerical_data.describe()

median_values = numerical_data.median().to_frame().T.rename(index={0: 'median'})
mode_values = numerical_data.mode().iloc[0].to_frame().T.rename(index={0: 'mode'})

complete_stats = pd.concat([descriptive_stats, median_values, mode_values])

complete_stats

In [ ]:
df.describe(include = [object])

Analýza vybraných kategorických atributů:

In [ ]:
df['artist(s)_name'].value_counts()

In [ ]:
# Zobrazení názvu skladeb, které se ve sloupci 'track_name' nacházejí více než jednou
df['track_name'].value_counts()

In [ ]:
# Filtrace pouze těch skupin, které mají více než jednoho interpreta
grouped = df.groupby('track_name')['artist(s)_name'].apply(list).reset_index()
grouped[grouped['artist(s)_name'].str.len() > 1]

In [ ]:
df['key'].value_counts()

In [ ]:
df['mode'].value_counts()

Z dat výše bylo zjištěno, že se některé sloupce chovají jako objekt, přestože obsahují číselná data, tudíž data budou upravena na numerické atributy. Některé atributy jsou rovněž neintuitivně pojmenovány a budou přejmenovány.

In [ ]:
attributes_to_numeric = ['streams', 'in_deezer_playlists', 'in_shazam_charts']

for attribute in attributes_to_numeric:
    df[attribute] = df[attribute].str.replace(',', '')
    df[attribute] = pd.to_numeric(df[attribute], errors='coerce')
    
df[attributes_to_numeric].dtypes

In [ ]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
numerical_data = df[numerical_cols]

descriptive_stats = numerical_data.describe()

median_values = numerical_data.median().to_frame().T.rename(index={0: 'median'})
mode_values = numerical_data.mode().iloc[0].to_frame().T.rename(index={0: 'mode'})

complete_stats = pd.concat([descriptive_stats, median_values, mode_values])

complete_stats

In [ ]:
df.describe(include = [object])

#### 2. Podúkol

Prozkoumejte rozložení hodnot jednotlivých atributů pomocí vhodných grafů, zaměřte se i na to, jak hodnota jednoho či dvou atributů ovlivní rozložení hodnot jiného atributu. Do dokumentace vložte alespoň 5 různých grafů, zobrazujících zjištěná rozložení hodnot. Použijte různé typy grafů (např. bodový graf, histogram, krabicový nebo houslový graf, graf složený z více podgrafů apod.) a věnujte se různým atributům. V dokumentaci také všechny grafy vhodně okomentujte – popište, jaké informace z nich můžeme vyčíst.

In [ ]:
# Vytvoření grafu zobrazujícího závislost 'streams' na 'mode'
plt.figure(figsize=(10, 6))
sns.boxplot(y='mode', x='streams', data=df, color='blue')
plt.title('Závislost počtu přehrání na módu skladeb')
plt.ylabel('Mód')
plt.xlabel('Počet přehrání')
plt.show()

In [ ]:
# Definování pořadí hudebních klíčů
keys = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

# Získání unikátních klíčů z dat
unique_keys = df['key'].unique()

# Seřazení unikátních klíčů podle hudební stupnice
ordered_keys = [key for key in keys if key in unique_keys]

# Vytvoření palety barev
palette = sns.color_palette("hls", len(ordered_keys))

plt.figure(figsize=(12, 8))
sns.boxplot(y="key", x="streams", data=df, order=ordered_keys, palette=palette, hue='key', hue_order=ordered_keys)
plt.xlabel("Počet přehrání")
plt.ylabel("Hudební klíč")
plt.title("Závislost počtu přehrání na hudebním klíči")
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(x="key", data=df, order=ordered_keys, palette=palette, hue='key', hue_order=ordered_keys)
plt.xlabel("Hudební klíč")
plt.ylabel("Počet")
plt.title("Počet jednotlivých hudebních klíčů")
plt.show()

In [ ]:
key_counts = df['key'].value_counts(normalize=True) * 100

ordered_key_counts = key_counts.reindex(ordered_keys)

ordered_key_counts.plot(kind='bar', color=palette)
plt.title('Procentuální rozložení hudebních klíčů dle skladeb')
plt.ylabel('Procento skladeb s hudebním klíčem [%]')
plt.xlabel('Hudební kíč')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
percentage_attributes = ['danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']

fig, axes = plt.subplots(7, 1, figsize=(20, 40))
axes = axes.flatten()

for i, attribute in enumerate(percentage_attributes):
    ax = axes[i]
    sns.scatterplot(x=attribute, y='streams', hue='key', data=df, ax=ax, hue_order=ordered_keys)
    ax.set_xlabel(f'Atribut {attribute}')
    ax.set_ylabel('Počet přehrání')
    ax.legend(title='Hudební klíč', loc='right')

plt.suptitle('Závislost počtu přehrání na jednotlivých procentuálních atributech dle hudebního klíče', y=1)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 40))

for i, key in enumerate(ordered_keys):
    key_data = df[df['key'] == key]
    plt.subplot(6, 2, i + 1)
    for attribute in percentage_attributes:
        plt.scatter(key_data[attribute], key_data['streams'], label=attribute, alpha=0.5)

    plt.xlabel('Hodnota atributu')
    plt.ylabel('Počet přehrání')
    plt.title(f'Počet přehrání proti hudebnímu klíči {key}')
    plt.legend(title='Procentuální atribut', loc='upper right')

plt.suptitle('Závislost počtu přehrání na hudebním klíči', y=1)
plt.tight_layout()
plt.show()

In [ ]:
for attribute in percentage_attributes:
    g = sns.FacetGrid(df, col='key', hue='key', hue_order=ordered_keys, col_wrap=3, height=4)
    g = g.map(sns.scatterplot, attribute, 'streams', alpha=0.5).add_legend()
    g.set_axis_labels(attribute, 'Počet přehrání')
    g.fig.suptitle(f'Závislost počtu přehrání proti atributu {attribute} podle hudebního Klíče')
    plt.xlim(0, 100)
    plt.subplots_adjust(top=0.95)
    plt.show()

In [ ]:
plt.figure(figsize=(15, 10)) 

for i, attribute in enumerate(percentage_attributes, 1):
    plt.subplot(3, 3, i)
    sns.histplot(data=df, x=attribute, bins=20, color='blue')
    plt.xlim(0, 100)
    plt.xlabel(attribute)
    plt.ylabel("Počet")
    plt.xlabel(f'Atribut {attribute}')

plt.suptitle('Počet hodnot dle procentuálních atributů')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, len(percentage_attributes) * 4))

for i, attribute in enumerate(percentage_attributes, 1):
    plt.subplot(len(percentage_attributes), 1, i)
    sns.violinplot(x=df[attribute], color='blue')
    plt.xlim(-15, 110)
    plt.xlabel(f'Atribut {attribute}')

plt.suptitle('Houslové grafy pro jednotlivé procentuální atributy', y=1)
plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(df[percentage_attributes], plot_kws={'color': 'blue'}, diag_kws={'color': 'blue'})
plt.suptitle('Závislost jednoho procentuálního atributu na ostatních', y=1)
plt.show()

In [ ]:
plt.figure(figsize=(15, 20)) 

for i, attribute in enumerate(percentage_attributes, 1):
    plt.subplot(4, 2, i)
    sns.scatterplot(x=attribute, y='streams', data=df, hue=attribute, palette='viridis', legend=False, alpha=0.5)
    plt.title(f'Závislost atributu {attribute} na počtu přehrání')
    plt.xlabel(attribute)
    plt.xlim(0, 100)
    plt.ylabel('Počet přehrání')
    plt.xlabel(f'Atribut {attribute}')

plt.suptitle('Závislost počtu přehrání na jednotlivých procentuálních atributech', y=1)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 20)) 

for i, attribute in enumerate(percentage_attributes, 1):
    plt.subplot(4, 2, i)
    sns.scatterplot(x=attribute, y='bpm', data=df, hue=attribute, palette='viridis', legend=False, alpha=0.5)
    plt.title(f'Závislost atributu {attribute} na BPM')
    plt.xlabel(attribute)
    plt.xlim(0, 100)
    plt.ylabel('BPM')
    plt.xlabel(f'Atribut {attribute}')

plt.suptitle('Závislost BPM na jednotlivých procentuálních atributech', y=1)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.histplot(df['bpm'], kde=True, color='blue')
plt.xlabel('BPM')
plt.ylabel('Počet')
plt.title('Histogram pro BPM')

plt.subplot(1, 2, 2)
sns.boxplot(x=df['bpm'], color='blue')
plt.xlabel('BPM')
plt.title('Krabicový graf pro BPM')

plt.show()

In [ ]:
plt.figure(figsize=(15, len(percentage_attributes) * 3))

for i, attribute in enumerate(percentage_attributes, 1):
    plt.subplot(len(percentage_attributes), 1, i)
    yearly_means = df.groupby('released_year')[attribute].mean()
    plt.plot(yearly_means, label=attribute, color='blue')
    plt.title(f'Atribut {attribute}')
    plt.xlabel('Rok vydání')
    plt.ylabel('Průměrná hodnota [%]')

plt.suptitle('Rozložení roku vydání skladby dle jednotlivých procentuálních atributů', y=1)
plt.tight_layout()
plt.show()

In [ ]:
year_counts = df['released_year'].value_counts().sort_index()

plt.figure(figsize=(15, 8))
sns.barplot(data=year_counts, color='blue')
plt.title('Rozložení skladeb podle roku vydání')
plt.xlabel('Rok vydání')
plt.xticks(rotation=45, ha='right')
plt.ylabel('Počet skladeb')
plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='artist_count', y='streams', data=df, color='blue')
plt.title('Vztah mezi počtem přehrání a počtem umělců ve skladbě')
plt.xlabel('Počet umělců')
plt.ylabel('Počet přehrání')
plt.show()

In [ ]:
plt.figure(figsize=(15, 20))

for i, attribute in enumerate(percentage_attributes, 1):
    plt.subplot(4, 2, i)
    sns.scatterplot(x=attribute, y='in_spotify_playlists', data=df, hue=attribute, palette='viridis', legend=False, alpha=0.5)
    plt.xlim(0, 100)
    plt.title(f"Závislost atributu {attribute} na počtu Spotify playlistů se skladbou")
    plt.xlabel(attribute)
    plt.ylabel("Počet Spotify playlistů se skladbou")

plt.suptitle('Rozložení počtu Spotify playlistů se skladbou dle jednotlivých procentuálních atributů', y=1)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 20))

for i, attribute in enumerate(percentage_attributes, 1):
    plt.subplot(4, 2, i)
    sns.scatterplot(x=attribute, y='in_apple_playlists', data=df, hue=attribute, palette='viridis', legend=False, alpha=0.5)
    plt.xlim(0, 100)
    plt.title(f"Závislost atributu {attribute} na počtu Apple playlistů se skladbou")
    plt.xlabel(attribute)
    plt.ylabel("Počet Apple playlistů se skladbou")

plt.suptitle('Rozložení počtu Apple playlistů se skladbou dle jednotlivých procentuálních atributů', y=1)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 20))

for i, attribute in enumerate(percentage_attributes, 1):
    plt.subplot(4, 2, i)
    sns.scatterplot(x=attribute, y='in_deezer_playlists', data=df, hue=attribute, palette='viridis', legend=False, alpha=0.5)
    plt.title(f"Závislost atributu {attribute} na počtu Deezer playlistů se skladbou")
    plt.xlabel(attribute)
    plt.ylabel("Počet Deezer playlistů se skladbou")

plt.suptitle('Rozložení počtu Deezer playlistů se skladbou dle jednotlivých procentuálních atributů', y=1)
plt.tight_layout()
plt.show()

In [ ]:
in_spotify_playlist = (df['in_spotify_playlists'] > 0).astype(int)
in_apple_playlist = (df['in_apple_playlists'] > 0).astype(int)
in_deezer_playlist = (df['in_deezer_playlists'] > 0).astype(int)

plt.figure(figsize=(15, 20))

for i, column in enumerate(percentage_attributes, 1):
    plt.subplot(4, 2, i)
    
    sns.kdeplot(df[in_spotify_playlist == 1][column], label='Spotify')
    sns.kdeplot(df[in_apple_playlist == 1][column], label='Apple')
    sns.kdeplot(df[in_deezer_playlist == 1][column], label='Deezer')

    plt.title(f"Distribuce {column}")
    plt.xlabel(column)
    plt.ylabel("Hustota")
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
in_spotify_chart = (df['in_spotify_charts'] > 0).astype(int)
in_apple_chart = (df['in_apple_charts'] > 0).astype(int)
in_deezer_chart = (df['in_deezer_charts'] > 0).astype(int)
in_shazam_chart = (df['in_shazam_charts'] > 0).astype(int)

plt.figure(figsize=(15, 20))

for i, column in enumerate(percentage_attributes, 1):
    plt.subplot(4, 2, i)
    
    sns.kdeplot(df[in_spotify_chart == 1][column], label='Spotify')
    sns.kdeplot(df[in_apple_chart == 1][column], label='Apple')
    sns.kdeplot(df[in_deezer_chart == 1][column], label='Deezer')
    sns.kdeplot(df[in_shazam_chart == 1][column], label='Shazam')

    plt.title(f"Distribuce {column}")
    plt.xlabel(column)
    plt.ylabel("Hustota")
    plt.legend()

plt.suptitle('Porovnání jednotlivých procentuálních atributů dle chartů', y=1)
plt.tight_layout()
plt.show()

#### 3. Podúkol

Zjistěte, zda zvolená datová sada obsahuje nějaké odlehlé hodnoty. V dokumentaci popište, jakým způsobem jste odlehlé hodnoty detekovali, a jaké hodnoty jste objevili.

In [ ]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

plt.figure(figsize=(15, 20))
for i, col in enumerate(numerical_cols):
    plt.subplot(len(numerical_cols)//3 + 1, 3, i+1)
    sns.histplot(x=df[col], color='blue')
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

plt.figure(figsize=(15, 20))
for i, col in enumerate(numerical_cols):
    plt.subplot(len(numerical_cols)//3 + 1, 3, i+1)
    sns.boxplot(x=df[col], color='blue')
    plt.title(col)
plt.tight_layout()
plt.show()

#### 4. Podúkol

Proveďte podrobnou analýzu chybějící hodnot. V dokumentaci popište celkový počet chybějících hodnot, počet objektů s více chybějícími hodnotami atd.

In [ ]:
missing_values = df.isnull().sum()

missing_values_df = pd.DataFrame(missing_values[missing_values > 0], columns=['Missing values'])
missing_values_df.index.name = 'Attribute'

total_rows = len(df)
missing_values_df['Percentage'] = ((missing_values_df['Missing values'] / total_rows) * 100).round(2)

missing_values_df

In [ ]:
df[df['streams'].isnull()]

In [ ]:
df[df['in_shazam_charts'].isnull()]

In [ ]:
df[df['key'].isnull()]

In [ ]:
# Výpočet počtu chybějících hodnot pro každý řádek
missing_per_row = df.isnull().sum(axis=1)

# Získání počtu řádků pro každý počet chybějících hodnot
row_counts = missing_per_row.value_counts().sort_index()

row_counts_df = pd.DataFrame(row_counts).reset_index()
row_counts_df.columns = ['Count of missing values', 'Number of rows']

row_counts_df

In [ ]:
# Filtrace pro zobrazení řádků s právě jednou chybějícími hodnotami
rows_with_one_missing = df[missing_per_row == 1]

rows_with_one_missing

In [ ]:
# Filtrace pro zobrazení řádků s právě dvěmi chybějícími hodnotami
rows_with_two_missing = df[missing_per_row == 2]

rows_with_two_missing

#### 5. Podúkol

Proveďte korelační analýzu numerických atributů (k analýze využijte grafy i korelační koeficienty).

In [ ]:
numerical_data = df.select_dtypes(include=[np.number])
correlation_matrix = numerical_data.corr()

correlation_matrix

In [ ]:
numerical_data = df.select_dtypes(include=[np.number])
display(numerical_data)
correlation_matrix = numerical_data.corr()

plt.figure(figsize=(20, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', square=True)
plt.xticks(rotation=45, ha='right')
plt.title('Korelační heatmapa numerických atributů')
plt.show()

In [ ]:
columns_to_correlate = ['streams', 'bpm', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']

correlation_matrix = df[columns_to_correlate].corr()

correlation_matrix

In [ ]:
columns_to_correlate = ['streams', 'bpm', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']

correlation_matrix = df[columns_to_correlate].corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.xticks(rotation=45, ha='right')
plt.title("Korelační heatmapa procentuálních hodnot a BPM")
plt.show()

In [ ]:
columns_to_correlate = ['streams', 'in_spotify_playlists', 'in_spotify_charts', 'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts']

correlation_matrix = df[columns_to_correlate].corr()

correlation_matrix

In [ ]:
columns_to_correlate = ['streams', 'in_spotify_playlists', 'in_spotify_charts', 'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts']

correlation_matrix = df[columns_to_correlate].corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.xticks(rotation=45, ha='right')
plt.title("Korelační mapa playlistů a chartů")
plt.show()

## 3. Úkol
### Zadání: 
Připravte 2 varianty datové sady vhodné pro dolovací algoritmy. Můžete uvažovat dolovací úlohu uvedenou u datové sady nebo navrhnout vlastní dolovací úlohy. V případě vlastní dolovací úlohy ji specifikujte v dokumentaci. V rámci přípravy datové sady proveďte následující kroky:
1. Z datové sady odstraňte atributy, které jsou pro danou dolovací úlohu irelevantní. V datové sadě, pokud možno, ponechte jak kategorické, tak i numerické atributy, atributy s chybějícími hodnotami a atributy s odlehlými hodnotami (pokud je původní datová sada obsahuje).
2. Vypořádejte se s chybějícími hodnotami. Pro odstranění těchto hodnot využijte alespoň dvě různé metody pro odstranění chybějících hodnot.
3. Vypořádejte se s odlehlými hodnotami, jsou-li v datové sadě přítomny.
4. Pro jednu variantu datové sady proveďte diskretizaci numerických atributů tak, aby výsledná datová sada byla vhodná pro algoritmy, které vyžadují na vstupu kategorické atributy.
5. Pro druhou variantu datové sady proveďte vhodnou transformaci kategorických atributů na numerické atributy. Dále pak proveďte normalizaci numerických atributů, které má smysl normalizovat. Výsledná datová sada by měla být vhodná pro metody vyžadující numerické vstupy.

### Řešení: 
#### 1. Podúkol:
Z datové sady odstraňte atributy, které jsou pro danou dolovací úlohu irelevantní. V datové sadě, pokud možno, ponechte jak kategorické, tak i numerické atributy, atributy s chybějícími hodnotami a atributy s odlehlými hodnotami (pokud je původní datová sada obsahuje).

In [ ]:
df = df.drop(['track_name', 'artist(s)_name', 'artist_count',
              'released_year', 'released_month', 'released_day',
              'in_spotify_playlists', 'in_spotify_charts', 'streams',
              'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists',
              'in_deezer_charts', 'in_shazam_charts', 'instrumentalness_%'], axis=1)
df.info()

#### 2. Podúkol:
Vypořádejte se s chybějícími hodnotami. Pro odstranění těchto hodnot využijte alespoň dvě různé metody pro odstranění chybějících hodnot.

In [ ]:
def remove_rows_missing_values(df, column_name):
    return df[df[column_name].notna()]

In [ ]:
print("Počet záznamů před odstraněním prázdných hodnot: ", len(df))

df = remove_rows_missing_values(df, "key")

print("Počet záznamů po odstraněním prázdných hodnot: ", len(df))

U vybrané datové sady byly ošetřeny chybějící hodnoty atributu "key". Pro ošetření hodnot byla vybrána metoda ignorování záznamu, jelikož nelze chybějící hodnotu doplnit (například nejčastější hodnotou).

Při přípravě dané datové sady pro dolovací úlohu nelze využít druhou metodu pro ošetření chybějících hodnot (doplnění chybějících hodnot ručně nebo automaticky), jelikož jediný atribut, který má chybějící hodnoty, je atribut "in_shazam_charts", který nebyl pro danou dolovací úlohu vybrán.

#### 3. Podúkol:
Vypořádejte se s odlehlými hodnotami, jsou-li v datové sadě přítomny.

In [ ]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [ ]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

plt.figure(figsize=(15, 20))
for i, col in enumerate(numerical_cols):
    plt.subplot(len(numerical_cols)//3 + 1, 3, i+1)
    sns.boxplot(x=df[col], color='blue')
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
df = remove_outliers(df, "bpm")
df = remove_outliers(df, "danceability_%")
df = remove_outliers(df, "valence_%")
df = remove_outliers(df, "energy_%")
df = remove_outliers(df, "acousticness_%")
df = remove_outliers(df, "liveness_%")
df = remove_outliers(df, "speechiness_%")

In [ ]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

plt.figure(figsize=(15, 20))
for i, col in enumerate(numerical_cols):
    plt.subplot(len(numerical_cols)//3 + 1, 3, i+1)
    sns.boxplot(x=df[col], color='blue')
    plt.title(col)
plt.tight_layout()
plt.show()

#### 4. Podúkol:
Pro jednu variantu datové sady proveďte diskretizaci numerických atributů tak, aby výsledná datová sada byla vhodná pro algoritmy, které vyžadují na vstupu kategorické atributy.

In [ ]:
def categorize_bpm(df):
    conditions = [
        (df['bpm'] <= 60),
        (df['bpm'] <= 80),
        (df['bpm'] <= 100),
        (df['bpm'] <= 120),
        (df['bpm'] <= 140),
        (df['bpm'] <= 160),
        (df['bpm'] > 160)
    ]

    categories = [
        'very_slow',
        'slow',
        'slightly_slow',
        'medium_tempo',
        'fast',
        'very_fast',
        'extremely_fast'
    ]

    df['category_bpm'] = np.select(conditions, categories, default='Not Categorized')
    df = df.drop(['bpm'], axis=1)

    return df

def categorize_percentage_values(df, column):
    conditions = [
        (df[column] <= 10),
        (df[column] <= 20),
        (df[column] <= 30),
        (df[column] <= 40),
        (df[column] <= 50),
        (df[column] <= 60),
        (df[column] <= 70),
        (df[column] <= 80),
        (df[column] <= 90),
        (df[column] <= 100),
    ]
    
    categories = [
        'btw_0_10',
        'btw_10_20',
        'btw_20_30',
        'btw_30_40',
        'btw_40_50',
        'btw_50_60',
        'btw_60_70',
        'btw_70_80',
        'btw_80_90',
        'btw_90_100'
    ]
    
    
    df['category_' + column] = np.select(conditions, categories, default='Not Categorized')
    df = df.drop([column], axis=1)
    
    return df

In [ ]:
df_categorize = df.copy()
df_categorize = categorize_bpm(df_categorize)

df_categorize = categorize_percentage_values(df_categorize, 'danceability_%')
df_categorize = categorize_percentage_values(df_categorize, 'valence_%')
df_categorize = categorize_percentage_values(df_categorize, 'energy_%')
df_categorize = categorize_percentage_values(df_categorize, 'acousticness_%')
df_categorize = categorize_percentage_values(df_categorize, 'liveness_%')
df_categorize = categorize_percentage_values(df_categorize, 'speechiness_%')

df_categorize.head(50).to_csv('categorize_data.csv', index=False)
df_categorize

#### 5. Podúkol:
Pro druhou variantu datové sady proveďte vhodnou transformaci kategorických atributů na numerické atributy. Dále pak proveďte normalizaci numerických atributů, které má smysl normalizovat. Výsledná datová sada by měla být vhodná pro metody vyžadující numerické vstupy.

In [ ]:
def transform_numeric(df, column):
    one_hot = pd.get_dummies(df[column])
    df = df.drop(column, axis = 1)
    return df.join(one_hot)

def normalize_column(df, column_name):
    df[column_name] = df[column_name] / 100
    return df

df_numeric = transform_numeric(df, 'key')
df_numeric = transform_numeric(df_numeric, 'mode')

df_numeric = normalize_column(df_numeric, 'danceability_%')
df_numeric = normalize_column(df_numeric, 'valence_%')
df_numeric = normalize_column(df_numeric, 'energy_%')
df_numeric = normalize_column(df_numeric, 'acousticness_%')
df_numeric = normalize_column(df_numeric, 'liveness_%')
df_numeric = normalize_column(df_numeric, 'speechiness_%')

df_numeric.head(50).to_csv('numeric_data.csv', index=False)
df_numeric